# TensorFlow Eager Execution Tutorial

TensorFlow *Get Started with Eager Execution* tutorial: https://www.tensorflow.org/get_started/eager

## Setup the Environment

In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

In [ ]:
print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

## Import the Training Dataset

In [ ]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

In [ ]:
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

In [ ]:
print(f'Local copy of the dataset file: {train_dataset_fp}')

In [ ]:
# Take a peak at the contents of the csv file
i = 0
with open(train_dataset_fp) as f:
    for line in f:
        print(line)
        i += 1
        
        if i > 5:
            break

## Parse the Training Dataset

In [ ]:
# Creating a function which will parse each line of the CSV file
def parse_csv(line):
  example_defaults = [[0.], [0.], [0.], [0.], [0]] # Sets the types expected
  parsed_line = tf.decode_csv(line, example_defaults)
  features = tf.reshape(parsed_line[:-1], shape=(4,))
  label = tf.reshape(parsed_line[-1], shape=())
  return features, label

In [ ]:
# TextLineDataset inherits from Dataset. It is a Dataset comprising lines from one or more text files.
train_dataset = tf.data.TextLineDataset(train_dataset_fp)

In [ ]:
# Skip the first line of the CSV, which contains information about the dataset
train_dataset = train_dataset.skip(1)

In [ ]:
# Maps a function across the dataset being held
# Function should map a nested structure of tensors (having shapes and types defined by self.output_shapes and self.output_types) to another nested structure of tensors.
train_dataset = train_dataset.map(parse_csv)

In [ ]:
# Randomly shuffles the elements of this dataset
# buffer_size: A tf.int64 scalar tf.Tensor, representing the number of elements from this dataset from which the new dataset will sample.
# buffer_size should be greater than the size of the dataset
train_dataset = train_dataset.shuffle(buffer_size=1000)

In [ ]:
# Combines consecutive elements of this dataset into batches.
train_dataset = train_dataset.batch(32)

In [ ]:
features, label = iter(train_dataset).next()
print("Example features:", features[0])
print("Example label:", label[0])

## Configure the Model

In [ ]:
# Linear stack of layers
# The activation function determines the output of a single neuron to the next layer. This is loosely based on how brain neurons are connected. 
# There are many available activations, but ReLU is common for hidden layers.
# Activation Function: A function (for example, ReLU or sigmoid) that takes in the weighted sum of all of the inputs from the previous layer and then generates and passes an output value (typically nonlinear) to the next layer.
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(3)
])

In [ ]:
# Defining a function to calculate the model's loss
# losses.sparse_softmax_cross_entropy will return a loss value that is progressively larger as the prediction gets worse
def loss(model, x, y):
  y_ = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

In [ ]:
# Defining a function to calculate the gradient which should be followed
# GradientTape: Record operations for automatic differentiation.
# Gradient: Computes the gradient using operations recorded in context of this tape.
# Model.variables: Returns the list of all layer variables/weights.
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

In [ ]:
# Applies the computed gradients to the model's variables to minimize the loss function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

## Train the Model

In [ ]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.variables),
                              global_step=tf.train.get_or_create_global_step())

    # Track progress
    epoch_loss_avg(loss(model, x, y))  # add current batch loss
    # compare predicted label to actual label
    epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

  # end epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  
  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

## Plot the Results of the Training

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)

plt.show()

## Test the Model

In [ ]:
test_url = "http://download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)

test_dataset = tf.data.TextLineDataset(test_fp)
test_dataset = test_dataset.skip(1)
test_dataset = test_dataset.map(parse_csv)
test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.batch(32)

In [ ]:
test_accuracy = tfe.metrics.Accuracy()

for (x, y) in test_dataset:
  prediction = tf.argmax(model(x), axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

## Make Predictions Using the Trained Model

In [ ]:
class_ids = ["Iris setosa", "Iris versicolor", "Iris virginica"]

predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  name = class_ids[class_idx]
  print("Example {} prediction {}: {}".format(i, logits, name))